In [1]:
import json
import os
from PIL import Image, ImageDraw
import torch
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import LLMChain
from langchain.prompts import PromptTemplate

/opt/anaconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('../phase1_results.json', 'r') as f:
    bounding_boxes = json.load(f)

In [3]:
image_folder = '../test_resized/'

In [4]:
output_folder = 'output_folder'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [5]:
def set_device():
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    return device

device=set_device()

In [6]:
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("stanford-crfm/BioMedLM")
model = AutoModelForCausalLM.from_pretrained("stanford-crfm/BioMedLM").to(device)

/opt/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
def preprocess_box(box, image_width, image_height):
    x_min, y_min, x_max, y_max = box
    width = x_max - x_min
    height = y_max - y_min
    position_x = x_min / image_width
    position_y = y_min / image_height
    box_center_x = (x_min + x_max) / 2
    box_center_y = (y_min + y_max) / 2
    
    return {
        "width": width,
        "height": height,
        "position_x": position_x,
        "position_y": position_y,
        "center_x": box_center_x,
        "center_y": box_center_y
    }

In [ ]:
def generate_prompt(box_data):
    prompt = "Based on the following abnormalities found in the X-ray image, please provide a detailed medical report:\n"
    for entry in box_data:
        box_info = preprocess_box(entry['box'], 512, 512)  # Assuming the image has been resized to 512x512
        prompt += (
            f"- {entry['class_label']} located at approximately "
            f"({box_info['position_x']:.2f}, {box_info['position_y']:.2f}) relative to the image size. "
            f"The bounding box is centered at ({box_info['center_x']:.2f}, {box_info['center_y']:.2f}).\n"
        )
    prompt += "What could be the potential diagnoses and implications?"
    return prompt

In [ ]:
def process_image(image_path, box_data):
    # Load and resize image
    image = Image.open(image_path).convert("RGB")
    image = image.resize((512, 512))
    
    # Draw bounding boxes
    draw = ImageDraw.Draw(image)
    for entry in box_data:
        box = entry['box']
        draw.rectangle(box, outline="red", width=3)
    
    return image

In [ ]:
image_ids_of_interest = ["f1ae5125f7a72feed87fc0afa919b261.png"]


In [ ]:
for image_id in image_ids_of_interest:
    if image_id not in bounding_boxes or not bounding_boxes[image_id]:  # Skip if no bounding boxes
        continue
    
    box_data = bounding_boxes[image_id]
    
    # Retrieve and process the image
    image_path = os.path.join(image_folder, image_id)
    processed_image = process_image(image_path, box_data)
    
    # Generate prompt with bounding box information
    prompt = generate_prompt(box_data)
    
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,      # Adjust as needed for concise reports
        temperature=0.2,         # Balance between creativity and coherence
        top_p=0.9,               # Nucleus sampling for diverse output
        repetition_penalty=1.5   # Penalty for repetition to avoid echoing the prompt
    )

    report = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Display the image with bounding boxes and the report
    # processed_image.show(title="X-ray Image")
    print(f"Report:\n{report}\n")

    # # Save the processed image with bounding boxes and report
    # save_path = os.path.join(output_folder, image_id)
    # processed_image.save(save_path)
    # with open(save_path.replace('.png', '_report.txt'), 'w') as report_file:
    #     report_file.write(report)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Report:
Based on the following abnormalities found in the X-ray image, please provide a detailed medical report:
- Consolidation located at approximately (0.38, 0.61) relative to the image size. The bounding box is centered at (310.00, 338.00).
- Atelectasis located at approximately (0.50, 0.27) relative to the image size. The bounding box is centered at (283.00, 172.00).
What could be the potential diagnoses and implications? 
The provided information indicates that there are two distinct findings present within this chest radiograph: consolidation of lung tissue with an associated increase density consistent with fluid or inflammation; as well as areas where air-filled structures have collapsed resulting from obstruction.
Consolidations can result due various causes including infections such pneumonia bacterial infection viral respiratory syncytial virus RSV tuberculosis TB bronchopulmonary dysplasia BPD aspiration foreign body injury trauma malignancy
Atelectases may arise secondary